# Составление фондового индекса

### Вы научитесь:

- работать с методом главных компонент
- использовать его для вычисления улучшенного индекса Доу-Джонса

### Введение

Метод главных компонент *(principal component analysis, PCA)* — это один из методов обучения без учителя, который позволяет сформировать новые признаки, являющиеся линейными комбинациями старых. При этом новые признаки строятся так, чтобы сохранить как можно больше дисперсии в данных. Иными словами, метод главных компонент понижает размерность данных оптимальным с точки зрения сохранения дисперсии способом.


Основным параметром метода главных компонент является количество новых признаков. Как и в большинстве методов машинного обучения, нет четких рекомендаций по поводу выбора значения этого параметров. Один из подходов — выбирать минимальное число компонент, при котором объясняется не менее определенной доли дисперсии (это означает, что в выборке сохраняется данная доля от исходной дисперсии).


В этом задании понадобится измерять схожесть двух наборов величин. Если имеется набор пар измерений (например, одна пара — предсказания двух классификаторов для одного и того же объекта), то охарактеризовать их зависимость друг от друга можно с помощью корреляции Пирсона. Она принимает значения от -1 до 1 и показывает, насколько данные величины линейно зависимы. Если корреляция равна -1 или 1, то величины линейно выражаются друг через друга. Если она равна нулю, то линейная зависимость между величинами отсутствует.

### Данные

В этом задании мы будем работать с данными о стоимостях акций 30 крупнейших компаний США. На основе этих данных можно оценить состояние экономики, например, с помощью индекса Доу-Джонса. Со временем состав компаний, по которым строится индекс, меняется. Для набора данных был взят период с *23.09.2013* по *18.03.2015*, в котором набор компаний был фиксирован (подробнее почитать о составе можно по ссылке из материалов).

Одним из существенных недостатков индекса Доу-Джонса является способ его вычисления — при подсчёте индекса цены входящих в него акций складываются, а потом делятся на поправочный коэффициент. В результате, даже если одна компания заметно меньше по капитализации, чем другая, но стоимость одной её акции выше, то она сильнее влияет на индекс. Даже большое процентное изменение цены относительно дешёвой акции может быть нивелировано незначительным в процентном отношении изменением цены более дорогой акции.

### Реализация в *sklearn*

Метод главных компонент реализован в пакете `scikit-learn` в модуле `decomposition` в классе `PCA`. Основным параметром является количество компонент (`n_components`). Для обученного преобразования этот класс позволяет вычислять различные характеристики. Например, поле `explained_variance_ratio_` содержит процент дисперсии, который объясняет каждая компонента. Поле `components_` содержит информацию о том, какой вклад вносят признаки в компоненты. Чтобы применить обученное преобразование к данным, можно воспользоваться методом `transform`.


Для нахождения коэффициента корреляции Пирсона можно воспользоваться функцией `corrcoef` из пакета `numpy`.

### Материалы

- [Dow Jones Industrial Average](https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average)
- [История состава компаний, входящих в индекс Dow Jones](https://en.wikipedia.org/wiki/Historical_components_of_the_Dow_Jones_Industrial_Average)

### Инструкция по выполнению

1. Загрузите данные `close_prices.csv`. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода.
2. На загруженных данных обучите преобразование `PCA` с числом компоненты равным 10. Скольких компонент хватит, чтобы объяснить 90% дисперсии?
3. Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.
4. Загрузите информацию об индексе Доу-Джонса из файла `djia_index.csv`. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?
5. Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [2]:
def answer(filename, *answer):
    file = open(filename, 'w')
    ans = ''
    for i in answer:
        ans += str(i) + ' '
    ans = ans[:-1]
    file.write(ans)
    file.close()

## 1
Загрузите данные `close_prices.csv`. В этом файле приведены цены акций 30 компаний на закрытии торгов за каждый день периода.

In [3]:
data = pd.read_csv('close_prices.csv')

In [4]:
data.head()

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 31 columns):
date    374 non-null object
AXP     374 non-null float64
BA      374 non-null float64
CAT     374 non-null float64
CSCO    374 non-null float64
CVX     374 non-null float64
DD      374 non-null float64
DIS     374 non-null float64
GE      374 non-null float64
GS      374 non-null float64
HD      374 non-null float64
IBM     374 non-null float64
INTC    374 non-null float64
JNJ     374 non-null float64
JPM     374 non-null float64
KO      374 non-null float64
MCD     374 non-null float64
MMM     374 non-null float64
MRK     374 non-null float64
MSFT    374 non-null float64
NKE     374 non-null float64
PFE     374 non-null float64
PG      374 non-null float64
T       374 non-null float64
TRV     374 non-null float64
UNH     374 non-null float64
UTX     374 non-null float64
V       374 non-null float64
VZ      374 non-null float64
WMT     374 non-null float64
XOM     374 non-null float

In [6]:
print(data.loc[0])

date    2013-09-23
AXP          76.44
BA          117.51
CAT          85.03
CSCO         24.27
CVX         125.52
DD           59.41
DIS          64.75
GE           24.28
GS          165.25
HD           75.91
IBM         190.99
INTC         23.62
JNJ          89.09
JPM          51.46
KO           38.63
MCD          97.28
MMM         121.11
MRK          47.68
MSFT         32.74
NKE          68.98
PFE           28.8
PG           79.28
T            34.22
TRV          86.38
UNH          71.82
UTX         109.42
V           196.24
VZ           47.98
WMT          76.42
XOM          87.75
Name: 0, dtype: object


In [8]:
useful_data = data.iloc[:, 1:]
useful_data.head()

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [9]:
useful_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 30 columns):
AXP     374 non-null float64
BA      374 non-null float64
CAT     374 non-null float64
CSCO    374 non-null float64
CVX     374 non-null float64
DD      374 non-null float64
DIS     374 non-null float64
GE      374 non-null float64
GS      374 non-null float64
HD      374 non-null float64
IBM     374 non-null float64
INTC    374 non-null float64
JNJ     374 non-null float64
JPM     374 non-null float64
KO      374 non-null float64
MCD     374 non-null float64
MMM     374 non-null float64
MRK     374 non-null float64
MSFT    374 non-null float64
NKE     374 non-null float64
PFE     374 non-null float64
PG      374 non-null float64
T       374 non-null float64
TRV     374 non-null float64
UNH     374 non-null float64
UTX     374 non-null float64
V       374 non-null float64
VZ      374 non-null float64
WMT     374 non-null float64
XOM     374 non-null float64
dtypes: float64(30)
memor

## 2
На загруженных данных обучите преобразование `PCA` с числом компоненты равным 10. Скольких компонент хватит, чтобы объяснить 90% дисперсии?

In [10]:
pca = PCA(n_components=10)

# transform n default features to m new ones
transformed_data = pca.fit_transform(useful_data)

# how many features explain 90% of dispersion?
component_dispersion = pca.explained_variance_ratio_
print(component_dispersion)

i = 1
while np.sum(component_dispersion[:i]) < 0.9:
    i += 1
print(i)
answer('submission-1.txt', i)

# Посчитав кумулятивные суммы, можно видеть, что 3 компоненты объясняют 89.9% дисперсии, а 4 - 92.77%

[ 0.73897118  0.11007169  0.04995088  0.0287492   0.02215448  0.01931577
  0.00674853  0.00614091  0.00320594  0.00305611]
4


In [11]:
print(transformed_data[0])

[-50.90240358  17.63167724   7.7360209    2.59600961   2.37756945
   4.24405501  12.8830885    3.55657041  -5.82197358  -1.72604005]


## 3
Примените построенное преобразование к исходным данным и возьмите значения первой компоненты.

In [12]:
first_component = transformed_data[:, 0]

## 4
Загрузите информацию об индексе Доу-Джонса из файла `djia_index.csv`. Чему равна корреляция Пирсона между первой компонентой и индексом Доу-Джонса?

In [13]:
djia_index = pd.read_csv('djia_index.csv')
corrcoef = np.corrcoef(first_component, djia_index.iloc[:, 1])[0, 1]
print(corrcoef)
answer('submission-2.txt', corrcoef)

# Можно увидеть, что коэффициент корреляции получился достаточно большим.

0.909652219305


## 5
Какая компания имеет наибольший вес в первой компоненте? Укажите ее название с большой буквы.

In [14]:
company = useful_data.keys()[pca.components_[0].argmax()]
print(company)
answer('submission-3.txt', company)

# Это сокращенное название компании Visa.

V
